In [10]:
import numpy as np
import pydot
import graphviz
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import AveragePooling2D, GlobalAveragePooling2D, Dense, Input
from tensorflow.keras.layers import Embedding, LSTM, Add
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [11]:
!pip install pydot

In [12]:
!pip install graphviz

# Get the X_train, y_train

In [13]:
import pickle
import numpy as np

## Load data for X1

### Import features file

In [14]:
features_file = '/Users/zhenliu15471/code/CMaxK/robo_romeo/raw_data/encoded_images_6k.pkl'

In [15]:
file = open(features_file, 'rb')
features_dict = pickle.load(file)
file.close()

In [16]:
features_dict['2513260012_03d33305cf'][0].shape

(8, 8, 1280)

## Create the X1_train

### Import captions data

In [17]:
captions_file = '/Users/zhenliu15471/code/CMaxK/robo_romeo/raw_data/cap'

In [18]:
file = open(captions_file, 'rb')
x2_captions = pickle.load(file)
file.close()

In [19]:
cap_img_list = x2_captions[0] # first column with list of repeated image id's
cap_img_list[0]

'1000268201_693b08cb0e'

### Create X1_train

In [21]:
X1_train = []
for cap_img in cap_img_list[:10000]:
    img_feature_matrix = features_dict[cap_img][0]
    X1_train.append(img_feature_matrix)
X1_train = np.array(X1_train)
X1_train.shape

(10000, 8, 8, 1280)

In [ ]:
ffffr

## Create X2_train

In [22]:
X2_train = np.array(x2_captions[1]).astype(np.uint32)[:10000]
X2_train.shape

(10000, 36)

In [23]:
x2_captions[2][13]

array([3])

In [24]:
X2_train[14]

array([  2,   1,  34,   3,   1,  72, 129,   5, 121,  57,   1, 473,  12,
       624,   3,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0], dtype=uint32)

## Create y_train

In [25]:
vocab_size = 7589

In [26]:
y_train = np.array([el[0] if len(el)>0 else vocab_size for el in x2_captions[2][:10000]])
y_train

array([  1,  34,   3, ...,  11,  93, 211])

In [27]:
x2_captions=None
del x2_captions

#  LSTM sequence model

## LSTM Model layers

In [28]:
max_caption_length = 36


In [29]:
inputs2  = Input(shape=(max_caption_length,),name="captions")
embed_layer = Embedding(vocab_size, 256, mask_zero=True)(inputs2)

input_encoded = Input(shape=(8,8,1280),name="images_encoded")

pooling = GlobalAveragePooling2D()(input_encoded)

cnn_dense = Dense(256, activation='relu')(pooling)


combine = Add()([embed_layer,cnn_dense])



2022-06-09 16:17:37.005008: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [30]:

lstm_layer = LSTM(256)(combine)
decoder = Dense(1000, activation='relu')(lstm_layer)
outputs = Dense(vocab_size, activation='softmax')(decoder)

##### Model summary Model summary

In [31]:

model = Model(inputs=[input_encoded, inputs2], outputs=outputs)
model.compile(loss='categorical_crossentropy' , optimizer='adam',
             metrics = 'accuracy')

In [32]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
images_encoded (InputLayer)     [(None, 8, 8, 1280)] 0                                            
__________________________________________________________________________________________________
captions (InputLayer)           [(None, 36)]         0                                            
__________________________________________________________________________________________________
global_average_pooling2d (Globa (None, 1280)         0           images_encoded[0][0]             
__________________________________________________________________________________________________
embedding (Embedding)           (None, 36, 256)      1942784     captions[0][0]                   
______________________________________________________________________________________________

In [33]:
plot_model(model,show_shapes=True)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


# Fit and train model 

In [34]:
import tensorflow as tf

In [35]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes=7589)

In [36]:
y_train.shape

(10000, 7589)

In [37]:
# The fit
model.fit(
x=(X1_train,X2_train),
    y=y_train,
    batch_size=32,
    epochs=10, 
    verbose=1
)

2022-06-09 16:18:00.679005: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
313/313 [==============================] - 23s 68ms/step - loss: 5.3570 - accuracy: 0.1509
Epoch 2/10
313/313 [==============================] - 22s 70ms/step - loss: 4.3783 - accuracy: 0.2349
Epoch 3/10
313/313 [==============================] - 21s 68ms/step - loss: 3.7784 - accuracy: 0.2858
Epoch 4/10
313/313 [==============================] - 21s 68ms/step - loss: 3.3168 - accuracy: 0.3200
Epoch 5/10
313/313 [==============================] - 21s 68ms/step - loss: 2.9282 - accuracy: 0.3659
Epoch 6/10
313/313 [==============================] - 21s 68ms/step - loss: 2.5816 - accuracy: 0.4038
Epoch 7/10
313/313 [==============================] - 21s 68ms/step - loss: 2.2485 - accuracy: 0.4378
Epoch 8/10
313/313 [==============================] - 21s 68ms/step - loss: 1.9496 - accuracy: 0.4778
Epoch 9/10
313/313 [==============================] - 22s 69ms/step - loss: 1.7007 - accuracy: 0.5174
Epoch 10/10
313/313 [==============================] - 21s 68ms/step - loss: 1.476

# Predict the results

In [39]:
CNN_model = EfficientNetB0(
    include_top=False, # Whether to include the fully-connected layer at the top of the network
    weights='imagenet', # pre-trained weights on ImageNet
    input_tensor=None,
    input_shape= (256,256,3), # It should have exactly 3 inputs channels
    pooling=None # Optional pooling mode for feature extraction when include_top is False
)

In [64]:
test_img_path = "../raw_data/Flicker8k_Dataset/2718495608_d8533e3ac5.jpg"

In [65]:
test_img= image.load_img(test_img_path, target_size=(256,256,3))
x_test = image.img_to_array(test_img)
x_test = np.expand_dims(x_test, axis=0)

In [66]:
test = CNN_model.predict(x_test)

In [67]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
inputs_seq='startsequence'
img_encoded=test
inputs_seq = [2]
# seq = 
for i in range(36):
    inputs_seq_model = pad_sequences([inputs_seq],padding='post',maxlen=36)
    y_pre = model.predict([img_encoded,inputs_seq_model])
    next_word = y_pre.argmax()
    if next_word == 13:
        break
    inputs_seq.append(next_word)

In [68]:
inputs_seq

[2, 1, 21, 177, 16, 3, 1, 1238, 3, 52, 12, 1, 442]

# Transfer the token to sentences

In [69]:
file ='../raw_data/word_index'

In [70]:
with open(file,'rb') as dictionary:
    b = pickle.load(dictionary)

In [75]:
sentence  = []

for number in inputs_seq:

    sentence.append(list(b.keys())[list(b.values()).index(number)])

In [77]:
sentence = sentence[1:]

In [83]:
final_sentence  = ' '.join(word for word in sentence)

In [84]:
final_sentence 

'a young asian boy in a pacifier in front of a railing'